<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Andrés Merino &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [17]:
# Paquetes necesarios
import kagglehub
import os  # Manejo de archivos
import numpy as np  # Manejo de datos
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización
import joblib
from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier


---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [2]:
# Leer los datos
path = kagglehub.dataset_download("harrywang/wine-dataset-for-clustering")
file_path = os.path.join(path, "wine-clustering.csv")
df = pd.read_csv(file_path)
# Muestro los primeros registros
display(df.head())

100%|██████████| 4.36k/4.36k [00:00<00:00, 7.27MB/s]

Extracting files...


,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735



### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [3]:
#Se escoge alcohol como la variable objetivo(se convierte en una variable categórica creando clases)
threshold = 13.5
df['Alcohol_Class'] = (df['Alcohol'] > threshold).astype(int)

# Dividir los datos en características (X) y objetivo (y)
X = df.drop(columns=['Alcohol', 'Alcohol_Class'])  # Características
y = df['Alcohol_Class']  # Variable objetivo

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [6]:
# Crear y entrenar un arbol con ganancia de información
modelo = DecisionTreeClassifier(criterion='entropy')
modelo.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [8]:
# Mostrar los hiperparámetros del modelo
print(modelo.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}


###Descripción Hiperparámetros

1. **`criterion`**: Este hiperparámetro determina la función de evaluación que se usará para dividir los nodos del árbol. Los dos valores más comunes son:
   - `'gini'`: Utiliza el índice de Gini para medir la impureza de los nodos.
   - `'entropy'`: Utiliza la entropía, que mide la incertidumbre en el nodo.

2. **`max_depth`**: Este parámetro limita la profundidad máxima del árbol. Si no se establece (es `None`), el árbol crecerá hasta que todos los nodos sean puros o hasta que todos los nodos contengan menos muestras que el mínimo requerido.

3. **`min_samples_split`**: Define el número mínimo de muestras que se requieren para dividir un nodo. Si un nodo tiene menos de este número de muestras, no se dividirá más. Es útil para evitar el sobreajuste (overfitting) en árboles muy grandes.

4. **`max_features`**: Especifica el número máximo de características a considerar al buscar la mejor división. Puede tomar valores como:
   - `None`: Utiliza todas las características.
   - `int`: Un número específico de características.
   - `float`: Un valor entre 0 y 1 que representa una fracción de las características.
   - `'sqrt'`: Utiliza la raíz cuadrada del número de características (esto es común en Random Forests).
   - `'log2'`: Utiliza el logaritmo en base 2 del número de características.

5. **`min_impurity_decrease`**: Este hiperparámetro controla el umbral mínimo de disminución en la impureza de un nodo para permitir una división. Si la disminución en la impureza es menor que este valor, el nodo no se dividirá más. Este parámetro puede ser útil para evitar que el árbol crezca demasiado y se sobreajuste a los datos.

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [10]:
# Definir los parámetros a optimizar
param_grid = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_impurity_decrease': [0.0, 0.01, 0.05],
    'max_features': [None, 'sqrt', 'log2']
}

In [11]:
# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, n_jobs=-1)

# Entrenar el modelo
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print(f"Mejores parámetros: {grid_search.best_params_}")

# Mejor modelo entrenado
mejor_modelo = grid_search.best_estimator_

# Evaluar el modelo con los datos de prueba
score = mejor_modelo.score(X_test, y_test)
print(f"Precisión en los datos de prueba: {score}")

Mejores parámetros: {'max_depth': 10, 'max_features': 'log2', 'min_impurity_decrease': 0.05, 'min_samples_split': 2}
Precisión en los datos de prueba: 0.8333333333333334


### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [13]:
#Definir los parametros
param_dist = {
    'max_depth': [None, 5, 10, 15, 20, 25],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10, 15, 20],  # Mínimo número de muestras para dividir un nodo
    'min_impurity_decrease': [0.0, 0.01, 0.05, 0.1, 0.2],  # Disminución mínima de la impureza
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.5]  # Número máximo de características a considerar en cada división
}

In [14]:
# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=modelo, param_distributions=param_dist,
                                   n_iter=25, cv=5, n_jobs=-1, random_state=42)

# Entrenar el modelo
random_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print(f"Mejores parámetros: {random_search.best_params_}")

# Mejor modelo entrenado
mejor_modelo = random_search.best_estimator_

# Evaluar el modelo con los datos de prueba
score = mejor_modelo.score(X_test, y_test)
print(f"Precisión en los datos de prueba: {score}")

Mejores parámetros: {'min_samples_split': 15, 'min_impurity_decrease': 0.05, 'max_features': 'sqrt', 'max_depth': 20}
Precisión en los datos de prueba: 0.8148148148148148


### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [15]:
# Crear el modelo con los mejores parámetros de GridSearchCV
modelo_optimo = DecisionTreeClassifier(max_depth=10,
                                       max_features='log2',
                                       min_impurity_decrease=0.05,
                                       min_samples_split=2,
                                       criterion='entropy')

# Reentrenar el modelo
modelo_optimo.fit(X_train, y_train)

# Evaluar el modelo con los datos de prueba
score_optimo = modelo_optimo.score(X_test, y_test)
print(f"Precisión con los mejores parámetros de GridSearchCV: {score_optimo}")

Precisión con los mejores parámetros de GridSearchCV: 0.8888888888888888


In [18]:
# Guardar el modelo entrenado
joblib.dump(modelo_optimo, 'modelo_optimo_gridsearch.joblib')

print("Modelo guardado como 'modelo_optimo_gridsearch.joblib'")

Modelo guardado como 'modelo_optimo_gridsearch.joblib'


### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>